In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/benjaminrike1/social_score_csr_reports.git

Cloning into 'social_score_csr_reports'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 259 (delta 14), reused 43 (delta 13), pack-reused 215
Receiving objects: 100% (259/259), 786.58 MiB | 21.22 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Checking out files: 100% (79/79), done.


In [3]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 5.4 MB/s 


In [4]:
!pip install tensorflow-text

     |████████████████████████████████| 4.6 MB 5.3 MB/s 
     |████████████████████████████████| 511.7 MB 3.7 kB/s 
     |████████████████████████████████| 438 kB 50.3 MB/s 
     |████████████████████████████████| 1.6 MB 26.9 MB/s 
     |████████████████████████████████| 5.8 MB 32.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: flatbuffers
  

In [5]:
pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 5.6 MB/s 
     |████████████████████████████████| 1.1 MB 37.7 MB/s 
     |████████████████████████████████| 47.8 MB 56 kB/s 
     |████████████████████████████████| 1.2 MB 36.6 MB/s 
     |████████████████████████████████| 352 kB 38.0 MB/s 
     |████████████████████████████████| 90 kB 7.4 MB/s 
     |████████████████████████████████| 99 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 10.3 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 237 kB 37.1 MB/s 


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import pickle5 as pickle

In [8]:
model_path = 'social_score_csr_reports/models/models/'
reloaded_model = tf.saved_model.load(model_path)


In [10]:
def print_my_examples(inputs, results):
  print("Probabilities is on the format: neutral, positive, negative")
  result_for_printing = \
    [f'input: {inputs[i]:<30} : \n Probability: Signal: {results[i][0]:.2f}    Noise: {results[i][1]:.2f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'In 2020, we continued to provide a framework of policies, standards, and implementation expectations to guide our supplier business relationships',
    'In 2020, we conducted risk assessments to help us identify and prioritize the most critical issue areas and suppliers. ',
    'We invest in charitable and community causes in all of our markets and our people regularly take part in volunteering activities to support social initiatives in our communities.',
    'As signatories of the Valuable 500 pledge, we are committed to putting disability on the business leadership agenda',
    'Our diversity was reduced this year',
    'Football is a sport',
    'In 2020, 35.6% of leadership positions were held by women, up from 35.5% in 2019. ',
    'Zero fatalities occurred during the year',
    'The launch of our cultural change CARE programme, which will embed ‘the seven health and safety habits’ within our organisation, was another success and', #positive
    '22% of our senior leadership team members are women', #negative?
    'Our Health and Safety Management System', #neutral
    'During the first quarter of 2021, we saw a spike in health and safety incidents',
    'Even though one uses generally the same letter “M” for both martingales and Markov process, these are a priori completely different processes!',
    'Not every process is either a sub- or a supermartingale',
    'This repository is for the exercises, homework and project. The course syllabus can be found here.',
    'Jeg heter Benjamin og skriver på norsk',
    'To establish the communication between the robot and the cloud via Wi-Fi, you need to find the IP address, as shown in Figure 3.',
    'We have drastically reduced our CO2 emission this year as we prepare to meet the latest regulatory demands.'
]
reloaded_results = tf.nn.softmax(reloaded_model(tf.constant(examples)))

print('Results from the model:')
print_my_examples(examples, reloaded_results)

Results from the model:
Probabilities is on the format: neutral, positive, negative
input: In 2020, we continued to provide a framework of policies, standards, and implementation expectations to guide our supplier business relationships : 
 Probability: Signal: 1.00    Noise: 0.00
input: In 2020, we conducted risk assessments to help us identify and prioritize the most critical issue areas and suppliers.  : 
 Probability: Signal: 1.00    Noise: 0.00
input: We invest in charitable and community causes in all of our markets and our people regularly take part in volunteering activities to support social initiatives in our communities. : 
 Probability: Signal: 1.00    Noise: 0.00
input: As signatories of the Valuable 500 pledge, we are committed to putting disability on the business leadership agenda : 
 Probability: Signal: 1.00    Noise: 0.00
input: Our diversity was reduced this year : 
 Probability: Signal: 1.00    Noise: 0.00
input: Football is a sport            : 
 Probability: Sign

In [14]:
filename = 'social_score_csr_reports/reports_df.pickle'
with open(filename, 'rb') as fh:
  data = pickle.load(fh)

data.drop(labels=['text'], inplace=True, axis=1)
data.head(5)

,Company,sentences
0,ABNAMRO,"[ABN AMRO’s business, strategy, performance a..."
1,AgroFair,"[SUSTAINABILITY REPORT 2020, - 03 -CONTENT IND..."
2,Amazon,[All In: Staying the Course on Our Commitment...
3,asr,"[Ditisdetijd, vandoen., Annual Report2020, ASR..."
4,bam,"[Integrated Report 2020, Royal BAM Group nv, R..."


In [15]:
import numpy as np
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

scores = [] #saving the individual company scores

# iterating over every company
for element in data['Company'].values:
  # extracting all sentences from the report
  features = data[data['Company']==element]['sentences'].values[0] 
  total_sentences = len(features)

  # variables for saving signal
  signal = 0

  # hitting the model with sentences in chunks to avoid RAM overload
  for chunk in chunks(features, 128):
      # calculating softmax for every sentence
      results = tf.nn.softmax(reloaded_model(tf.constant(chunk)))
      # argmax to extract label
      argmax = tf.math.argmax(results, axis = 1).numpy()

      #counting pos sentences
      signal += np.count_nonzero(argmax==0)
  # assigning total scores realtive to number of sentences
  score = (signal)/total_sentences
  scores.append(score)

In [17]:
data['scores'] = scores

In [52]:
data['scores'] = data['scores'].apply(lambda x: 1-x)

In [56]:
data[['Company', 'scores']].to_csv('drive/MyDrive/ml/social_scores.csv')

In [57]:
for i,element in enumerate(data['scores']):
  print("%s: %.3f"%(data['Company'].values[i],element))

ABNAMRO: 0.254
AgroFair: 0.290
Amazon: 0.322
asr: 0.280
bam: 0.381
CocaCola: 0.318
Dataflex: 0.321
DS-Smith: 0.293
Essity: 0.364
Facebook: 0.339
Forbo: 0.300
Google: 0.189
GrandVision: 0.467
Grolsch: 0.248
heineken: 0.307
HEMA: 0.401
Henkel: 0.337
HP: 0.332
ING: 0.332
Intel: 0.306
KPN: 0.297
Microsoft: 0.165
NNgroup: 0.319
Optigroup: 0.284
PepsiCo: 0.326
Philips: 0.367
ProcterandGamble: 0.462
Renewi: 0.274
Rockwool: 0.283
Royal-Avebe: 0.368
Shell: 0.189
tarkett: 0.390
Tesla: 0.248
Trivium: 0.233
VanLanschot: 0.413
